# Opportunity

***Opportunity:*** Distance beyond 300 m (pressure) multiplied by the population sum

To do this, we need:

- `vector` population layer. Centroids, not polygons
- `raster` reference layer for resolution and crs
- `raster` layer for pressure 

#### Import libraries


In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import time
from rasterio import features
from rasterio.plot import show
from rasterio import plot

#### Import data

In [2]:
%%time
# Read in the vector data (OGS) and reference raster 
Pop = gpd.read_file("P:/NERC_07901_DEFRAG 2021/GIS/DO/Data/All/Pop/all_pop_centroids_E_EM_SE_2km_buffer_from_built_sites.gpkg")
Ref_Ras = rasterio.open("P:/NERC_07901_DEFRAG 2021/GIS/DO/Data/All/Raster/reference_raster_gspaces_3km_from_built_sites.tif")
Press = rasterio.open("../S1_pressure/pressure.tif")

CPU times: total: 30.7 s
Wall time: 1min 28s


#### Rasterize pop data - Sum

In [3]:
%%time
# Get list of geometries and population values from vector layer
geom = [geom for geom in Pop.geometry]
pop_values = Pop['Population'].tolist()

# Create a generator of (geometry, value) pairs
shapes = ((geom, value) for geom, value in zip(geom, pop_values))

# Rasterize the Pop layer and sum population values within each cell
Pop_Rast = features.rasterize(
    shapes=shapes,
    out_shape=Ref_Ras.shape,
    transform=Ref_Ras.transform,
    fill=0,
    merge_alg=rasterio.enums.MergeAlg.add
)

CPU times: total: 1.75 s
Wall time: 2.98 s


#### Opportunity

In [6]:
# Read in the pressure raster layer as a numpy array
Press_np = Press.read(1)

In [16]:
# Calculate opportunity
opp = Pop_Rast * Press_np

#### Save Output

In [7]:
# Save this layer as new raster file as it is currently a numpy array (has no refernce to coordinates).
# We will want to save the file and assign it coordinates and a crs. Easiest way to do this is to use a reference raster layer (e.g., Ref Raster)

# Copy the profile of Pop layer (e.g. CRS, extent, etc.)
profile_ref = Ref_Ras.profile

# Write the file
with rasterio.open("opportunity.tif", 'w', **profile_ref) as dst:
     dst.write(opp, 1)